# Setup

In [1]:
from db_funcs import get_all_content, get_db_con
import dotenv
dotenv.load_dotenv()
conn=get_db_con()

# Article Chunk Size
Determine what kinda size the current article chunks are, decided that for now, chunking is unnecesary

In [2]:
import numpy as np
import random
with conn.cursor() as cur:
    all_content = get_all_content(cur)
content_lengths = [len(content) for content in all_content]

print(f"Average length: {np.mean(content_lengths):.2f}")
print(f"Maximum length: {max(content_lengths)}")
print(f"Minimum length: {min(content_lengths)}")
print(f"Standard deviation: {np.std(content_lengths):.2f}")
print(f"\nSample of lengths: {random.sample(content_lengths, min(20, len(content_lengths)))}")

Average length: 802.61
Maximum length: 4063
Minimum length: 169
Standard deviation: 359.58

Sample of lengths: [488, 959, 636, 1020, 1305, 754, 585, 612, 1510, 407, 668, 874, 641, 599, 1018, 803, 535, 585, 753, 1463]


# Extract Sentances and Entity Relationships

In [ ]:
from transformers import pipeline
from utils import extract_triplets, Sentence_Extractor
from neo4j_funcs import Neo4j_Driver
from typing import List
from models import triplet

sentence_extractor=Sentence_Extractor()
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

neo4j = Neo4j_Driver()

sentences=[]

for content in all_content:
    sentences.append(sentence_extractor.get_sentences(content))

extracted_triplets:List[triplet]=[]
extracted_tokens=triplet_extractor(sentences[0:10], return_tensors=True, return_text=False)
for extracted_token in extracted_tokens:
    batch=[extracted_token[0]["generated_token_ids"]]
    extracted_text = triplet_extractor.tokenizer.batch_decode(batch)
    sentence_triplets = extract_triplets(extracted_text[0])
    extracted_triplets+= sentence_triplets

for e in extracted_triplets :
    neo4j.insert_relationship(e.head, e.type, e.type)
    

[nltk_data] Downloading package punkt_tab to /home/chris/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Device set to use cuda:0
